In [ ]:
import pickle as pkl

In [ ]:
with open('','rb') as f:
    Q = pkl.load(f)

In [ ]:
with open('','rb') as f:
    U = pkl.load(f)

In [ ]:
class Question_Ans(Dataset):
    def __init__(self, filename, mode='train'):
        self.df = pd.read_csv(filename)
        self.questionid = self.df['QuestionId'].values
        self.userid = self.df['UserId'].values
        self.ans = self.df['IsCorrect'].values
        
        self.ans = 2*self.ans - 1
        
        self.length=len(self.ans)
        
        
        if(mode=='train'):
            start=int(0*self.length)
            end=int(0.8*self.length)
        elif(mode=='val'):
            start=int(0.8*self.length)
            end=int(1*self.length)
        else:
            start = 0
            end=self.length
            
            
        self.questionid = self.questionid[start:end]
        self.userid = self.userid[start:end]
        self.ans = self.ans[start:end]
        
        self.length=len(self.ans)
        
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        
        qid = self.questionid[idx]
        uid = self.userid[idx]
        ans = self.ans[idx]
        return qid,uid,ans

In [ ]:
base_dir = '../Data/'

input_dir = base_dir + "data/train_data"
train_data = base_dir + "data/train_data/train_task_1_2.csv"
test_data_1 = base_dir + "data/test_data/test_public_answers_task_1.csv"

In [ ]:
# df_test = pd.read_csv(test_data_1)

In [ ]:
test_dataset = Question_Ans(filename=test_data_1,mode='test')

In [ ]:
test_dataloader = torch.utils.data.DataLoader(test_dataset,batch_size=batch_size, shuffle=True)

In [ ]:
def get_qvector(questions):
    ans = Q(torch.LongTensor(questions))
    return ans

In [ ]:
def get_uvector(users):
    ans = U(torch.LongTensor(users)) 
    return ans

In [ ]:
def get_score(qvectors,uvectors):
    q_unsq = torch.unsqueeze(qvectors, 1)
    u_unsq = torch.unsqueeze(uvectors, 2)
    score = torch.bmm(q_unsq,u_unsq)
    score = torch.squeeze(score)
    return score

In [ ]:
for questions,users,ans in tqdm(dataloader[phase]):
    ans = torch.tensor(ans)
    ans = ans.type(dtype)

    qvectors = get_qvector(questions)
    uvectors = get_uvector(users)
    scores = get_score(qvectors,uvectors)

    if phase == 'train':
        loss.backward()
        optimizer.step()

    # statistics
    running_loss += loss.item()